In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("mustang_release_v0.2.0.csv")
df.head()

,user_ID,group_ID,submit_time,start_time,end_time,wallclock_limit,job_status,node_count,tasks_requested
0,351,354,2011-10-27 10:50:10-06:00,2011-10-27 11:49:02-06:00,2011-10-27 11:50:36-06:00,365 days 00:00:00.000000000,CANCELLED,1,24
1,351,354,2011-10-27 11:50:46-06:00,2011-10-27 11:51:06-06:00,2011-10-27 11:51:13-06:00,365 days 00:00:00.000000000,COMPLETED,1,24
2,287,288,2011-10-27 12:02:50-06:00,2011-10-27 12:03:08-06:00,2011-10-27 12:03:13-06:00,365 days 00:00:00.000000000,COMPLETED,1,24
3,287,288,2011-10-27 12:02:29-06:00,2011-10-27 12:02:36-06:00,2011-10-27 12:03:14-06:00,365 days 00:00:00.000000000,COMPLETED,1,24
4,351,354,2011-10-27 12:02:26-06:00,2011-10-27 12:02:36-06:00,2011-10-27 12:03:18-06:00,365 days 00:00:00.000000000,COMPLETED,800,19200


In [3]:
df = df.dropna()
df = df[df['job_status'] != 'CANCELLED']

# Convert the 'submit_time' and 'start_time' columns to datetime
df['submit_time'] = pd.to_datetime(df['submit_time'], utc=True)
df['start_time'] = pd.to_datetime(df['start_time'], utc=True)
df['end_time'] = pd.to_datetime(df['end_time'], utc=True)

# Calculate the wait time and run time in seconds
df['wait_time'] = (df['start_time'] - df['submit_time']).dt.total_seconds()
df['run_time'] = (df['end_time'] - df['start_time']).dt.total_seconds()

In [4]:
df_job = df[["wait_time", "run_time", "node_count"]]
df_job.head()

,wait_time,run_time,node_count
1,20.0,7.0,1
2,18.0,5.0,1
3,7.0,38.0,1
4,10.0,42.0,800
5,2.0,8.0,1


In [5]:
rows_with_zeros = (df_job == 0).any(axis=1)
print(df_job[rows_with_zeros])

         wait_time  run_time  node_count
1262           0.0    1769.0          64
1302           0.0    1790.0          64
1765        2105.0       0.0           0
1766          43.0       0.0           0
1767          27.0       0.0           0
...            ...       ...         ...
2100461        9.0       0.0           0
2100574        3.0       0.0           0
2104021        4.0       0.0           0
2104612        3.0       0.0           0
2105331        7.0       0.0           0

[23537 rows x 3 columns]


In [6]:
df_job = df_job.loc[~(df_job==0).any(axis=1)]
df_job.to_csv('mustang_data.csv', index=False)